In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_out = pd.read_csv("test.csv")

In [3]:
test.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
def data_clean(ds):
    ds['Embarked'].replace({'S':1, 'C':2, 'Q':3}, inplace = True)
    ds['Sex'].replace({'male':0,'female':1}, inplace = True)
    ds.drop(['PassengerId','Ticket', 'Fare', 'Cabin', 'Name'], axis=1,inplace=True)
    ds["Age"] = ds.groupby(['Sex','Pclass','SibSp','Parch'])['Age'].transform(lambda x: x.fillna(x.mean()))
    ds["Age"] = ds.groupby(['Sex','Pclass'])['Age'].transform(lambda x: x.fillna(x.mean()))
    ds["Embarked"] = ds.groupby(['Sex','Pclass'])['Embarked'].transform(lambda x: x.fillna(x.median()))
    return ds

In [5]:
train = data_clean(train)
test = data_clean(test)

In [6]:
train.head(50)

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,0,22.000000,1,0,1.0
1,1,1,1,38.000000,1,0,2.0
2,1,3,1,26.000000,0,0,1.0
3,1,1,1,35.000000,1,0,1.0
4,0,3,0,35.000000,0,0,1.0
5,0,3,0,29.184492,0,0,3.0
6,0,1,0,54.000000,0,0,1.0
7,0,3,0,2.000000,3,1,1.0
8,1,3,1,27.000000,0,2,1.0
9,1,2,1,14.000000,1,0,2.0


In [7]:
X = train.drop(['Survived'], axis='columns')
y = train['Survived']

In [8]:
test[test.isnull().any(axis=1)]

,Pclass,Sex,Age,SibSp,Parch,Embarked


In [9]:
##Separando dataset para treino e teste
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=42)

In [10]:
##Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model1 = LogisticRegression()
model1.fit(X_train, y_train)
pred = model1.predict(X_test)
log_acc = accuracy_score(pred, y_test)
print(log_acc)

0.8666666666666667


In [11]:
##RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
model2=RandomForestClassifier(n_estimators =100)
model2.fit(X_train,y_train)
pred_2=model2.predict(X_test)
rcf=accuracy_score(pred_2,y_test);
print(rcf)

0.8444444444444444


In [12]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
model4=KNeighborsClassifier(n_neighbors=9)
model4.fit(X_train,y_train)
pred_4=model4.predict(X_test)
knn=accuracy_score(pred_4,y_test);
print(knn)

0.7777777777777778


C:\Users\Danilo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [13]:
test.head()

,Pclass,Sex,Age,SibSp,Parch,Embarked
0,3,0,34.5,0,0,3
1,3,1,47.0,1,0,1
2,2,0,62.0,0,0,3
3,3,0,27.0,0,0,1
4,3,1,22.0,1,1,1


In [14]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Embarked
0,3,0,22.0,1,0,1.0
1,1,1,38.0,1,0,2.0
2,3,1,26.0,0,0,1.0
3,1,1,35.0,1,0,1.0
4,3,0,35.0,0,0,1.0


In [15]:
model1.fit(X,y)
pred = model1.predict(test) #Logistic Regression

In [16]:
sub = pd.DataFrame({"PassengerId": test_out["PassengerId"],"Survived":pred})

sub.to_csv('sub.csv',index=False)

In [17]:
test.groupby(['Sex']).mean()

,Pclass,Age,SibSp,Parch,Embarked
Sex,,,,,
0,2.334586,29.664514,0.379699,0.274436,1.398496
1,2.144737,29.610879,0.565789,0.598684,1.578947
